<a href="https://colab.research.google.com/github/detel/Algorithms/blob/master/Chapter_2_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0+cu124
tiktoken version: 0.9.0


In [ ]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
import regex as re

class SimpleTokenizerV1:
    def encode(self, text):
        # Implement encode
        raise NotImplementedError

    def decode(self, ids):
        # Implement decode
        raise NotImplementedError

In [ ]:
import unittest
import os
import urllib.request

class TestSimpleTokenizerV1(unittest.TestCase):

    def prepare_the_verdict_vocab(self):
        if not os.path.exists("the-verdict.txt"):
          url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
          file_path = "the-verdict.txt"
          urllib.request.urlretrieve(url, file_path)
        with open("the-verdict.txt", "r", encoding="utf-8") as f:
          raw_text = f.read()
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        all_words = sorted(set(preprocessed))
        return {token:integer for integer,token in enumerate(all_words)}

    def setUp(self):
        self.hello_world_vocab = {"hello": 1, " ": 2, "world": 3, "!": 4}
        self.the_verdict_vocab = self.prepare_the_verdict_vocab()

    def test_init_success(self):
        # Test case 1: initialize of tokenizer
        tokenizer = SimpleTokenizerV1(self.hello_world_vocab)

    def test_encode_helloWorld(self):
        tokenizer = SimpleTokenizerV1(self.hello_world_vocab)

        # Test case 1: simple sentence
        text = "hello world"
        self.assertEqual(tokenizer.encode(text), [1, 3])

    def test_encode_helloWorldWithPunctuation(self):
        tokenizer = SimpleTokenizerV1(self.hello_world_vocab)

        text = "hello world "
        self.assertEqual(tokenizer.encode(text), [1, 3])

    def test_encode_emptyString(self):
        tokenizer = SimpleTokenizerV1(self.hello_world_vocab)

        self.assertEqual(tokenizer.encode(""), [])

    def test_encode_longStatement(self):
        tokenizer = SimpleTokenizerV1(self.the_verdict_vocab)

        text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
        ids = tokenizer.encode(text)
        self.assertEqual(ids, [1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7])

    def test_decode_helloWorld(self):
        tokenizer = SimpleTokenizerV1(self.hello_world_vocab)
        self.assertEqual(tokenizer.decode([1, 3]), "hello world")

    def test_decode_theVerdict(self):
        tokenizer = SimpleTokenizerV1(self.the_verdict_vocab)
        self.assertEqual(
            tokenizer.decode([1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]),
           '" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.')

    def test_encodeDecode_theVerdict(self):
        tokenizer = SimpleTokenizerV1(self.the_verdict_vocab)
        self.assertEqual(
            tokenizer.decode(tokenizer.encode(
                '"It\'s the last he painted, you know," Mrs. Gisburn said with pardonable pride.')),
            '" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.')

    def test_encode_stringWithCharactersNotInHelloWorldVocabFails(self):
        tokenizer = SimpleTokenizerV1(self.hello_world_vocab)

        with self.assertRaises(KeyError):
            tokenizer.encode("hello world?")

    def test_encode_stringWithCharactersNotInVerdictVocabFails(self):
        tokenizer = SimpleTokenizerV1(self.the_verdict_vocab)

        with self.assertRaises(KeyError):
            tokenizer.encode("Hello, do you like tea. Is this-- a test?")

# To run the tests in a Colab notebook:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.113s

OK


In [ ]:
import unittest
import os
import urllib.request
import regex as re

class TestSimpleTokenizerV2(unittest.TestCase):

    def prepare_the_verdict_vocab(self):
        if not os.path.exists("the-verdict.txt"):
            url = ("https://raw.githubusercontent.com/rasbt/"
                   "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
                   "the-verdict.txt")
            file_path = "the-verdict.txt"
            urllib.request.urlretrieve(url, file_path)
        with open("the-verdict.txt", "r", encoding="utf-8") as f:
            raw_text = f.read()
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
        preprocessed = [item.strip() for item in preprocessed
                        if item.strip()]
        all_words = sorted(set(preprocessed))
        return {token:integer for integer,token in enumerate(all_words)}

    def setUp(self):
        self.hello_world_vocab = {"hello": 1, " ": 2, "world": 3, "!": 4}
        self.the_verdict_vocab = self.prepare_the_verdict_vocab()

    # def test_encode_stringWithCharactersNotInHelloWorldVocabFails(self):
    #     tokenizer = SimpleTokenizerV2(self.hello_world_vocab)

    #     with self.assertRaises(KeyError):
    #         tokenizer.encode("hello world?")

    # def test_encode_stringWithCharactersNotInVerdictVocabFails(self):
    #     tokenizer = SimpleTokenizerV2(self.the_verdict_vocab)

    #     with self.assertRaises(KeyError):
    #         tokenizer.encode("Hello, do you like tea. Is this-- a test?")

    def test_encode_stringWithCharactersNotInVerdictVocabFails(self):
        tokenizer = SimpleTokenizerV2(self.the_verdict_vocab)
        text1 = "Hello, do you like tea?"
        text2 = "In the sunlit terraces of the palace."
        text = " <|endoftext|> ".join((text1, text2))

        self.assertEquals(tokenizer.encode(text),
                          [1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988,
                           956, 984, 722, 988, 1131, 7])

    def test_decode_stringWithCharactersNotInVerdictVocabFails(self):
        tokenizer = SimpleTokenizerV2(self.the_verdict_vocab)
        text1 = "Hello, do you like tea?"
        text2 = "In the sunlit terraces of the palace."
        text = " <|endoftext|> ".join((text1, text2))

        self.assertEquals(tokenizer.decode([1131, 5, 355, 1126, 628, 975, 10,
                                            1130, 55, 988, 956, 984, 722, 988,
                                            1131, 7]),
                          '<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'
)

        # add encode decode test


# To run the tests in a Colab notebook:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.502s

OK


In [ ]:
import regex as re

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {v: k for k, v in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
import regex as re

class SimpleTokenizerV2:
    def __init__(self, vocab):
        vocab.update({"<|endoftext|>": len(vocab)})
        vocab.update({"<|unk|>": len(vocab)})
        self.str_to_int = vocab
        self.int_to_str = {v: k for k, v in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

tiktoken version: 0.9.0


[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
import torch

class TestDataLoaderV1(unittest.TestCase):

    def setUp(self):
        with open("the-verdict.txt", "r", encoding="utf-8") as f:
            raw_text = f.read()
        self.dataloader = create_dataloader_v1(
            raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

    def test_first_batch(self):
        data_iter = iter(self.dataloader)
        first_batch = next(data_iter)

        self.assertEqual(len(first_batch), 2) # Should return input_ids and target_ids
        self.assertTrue(torch.equal(first_batch[0], torch.tensor([[  40,  367, 2885, 1464]]))) # input_ids shape
        self.assertTrue(torch.equal(first_batch[1], torch.tensor([[ 367, 2885, 1464, 1807]]))) # target_ids shape

    def test_second_batch(self):
        dataloader_2 = create_dataloader_v1(
            raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
        data_iter = iter(dataloader_2)
        first_batch = next(data_iter)
        second_batch = next(data_iter)

        self.assertEqual(len(second_batch), 2) # Should return input_ids and target_ids
        self.assertTrue(torch.equal(second_batch[0], torch.tensor([[ 367, 2885, 1464, 1807]]))) # input_ids shape
        self.assertTrue(torch.equal(second_batch[1], torch.tensor([[ 2885, 1464, 1807, 3619]]))) # target_ids shape

    def test_large_batch_stride(self):
        dataloader_2 = create_dataloader_v1(
            raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
        data_iter = iter(dataloader_2)
        input, target = next(data_iter)

        self.assertTrue(torch.equal(input, torch.tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]))) # input_ids shape
        self.assertTrue(torch.equal(target, torch.tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]]))) # target_ids shape

    # def test_dataloader_overlap(self):
    #     max_length = 5
    #     stride = 1
    #     dataloader = create_dataloader_v1(
    #         self.sample_text, batch_size=1, max_length=max_length,
    #         stride=stride, shuffle=False, drop_last=False, num_workers=0
    #     )

    #     data_list = list(dataloader)

    #     # Check if the sequences are overlapping correctly based on stride
    #     if len(data_list) > 1:
    #         input_ids_1 = data_list[0][0].squeeze()
    #         input_ids_2 = data_list[1][0].squeeze()
    #         self.assertTrue(torch.equal(input_ids_1[stride:], input_ids_2[:-stride]))

    # def test_dataloader_target_shift(self):
    #     max_length = 5
    #     stride = 2
    #     dataloader = create_dataloader_v1(
    #         self.sample_text, batch_size=2, max_length=max_length,
    #         stride=stride, shuffle=False, drop_last=False, num_workers=0
    #     )

    #     data_iter = iter(dataloader)
    #     first_batch = next(data_iter)

    #     input_ids = first_batch[0]
    #     target_ids = first_batch[1]

    #     # Check if the target is the input shifted by one token
    #     self.assertTrue(torch.equal(input_ids[:, 1:], target_ids[:, :-1]))

    # def test_dataloader_drop_last(self):
    #     # Create a text where the last batch would be incomplete if drop_last is True
    #     small_text = "abc"
    #     batch_size = 2
    #     max_length = 1

    #     dataloader_drop_last_true = create_dataloader_v1(
    #         small_text, batch_size=batch_size, max_length=max_length,
    #         stride=1, shuffle=False, drop_last=True, num_workers=0
    #     )
    #     # Expected number of batches with drop_last=True: floor(len(dataset) / batch_size)
    #     # len(dataset) is approx len(tokenized_text) - max_length.
    #     # For small_text="abc", max_length=1, len(tokenized_text) approx 3, len(dataset) approx 2.
    #     # floor(2 / 2) = 1
    #     self.assertEqual(len(list(dataloader_drop_last_true)), 1)

    #     dataloader_drop_last_false = create_dataloader_v1(
    #         small_text, batch_size=batch_size, max_length=max_length,
    #         stride=1, shuffle=False, drop_last=False, num_workers=0
    #     )
    #     # Expected number of batches with drop_last=False: ceil(len(dataset) / batch_size)
    #     # ceil(2 / 2) = 1
    #     self.assertEqual(len(list(dataloader_drop_last_false)), 1)


# To run the tests in a Colab notebook:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.576s

OK


In [ ]:
import regex as re

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {v: k for k, v in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [ ]:
import regex as re

class SimpleTokenizerV2:
    def __init__(self, vocab):
        vocab.update({"<|endoftext|>": len(vocab)})
        vocab.update({"<|unk|>": len(vocab)})
        self.str_to_int = vocab
        self.int_to_str = {v: k for k, v in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text